# Position and Velocity from Orbital Elements
##### Vallado Algorithm 10 (from pg 118)
##### COE2RV(p, e, i, capOmega, omega, nu (lambda_true, mu, omegaHat_true) --> r, v)

****
Inputs: 

* p: semiparameter (km)   (SIZE)
* e: eccentricity          (SHAPE)
* i: inclination             (TILT)
* capOmega: longitude of ascending node  (PIN)
* omega: argument of periapsis     (TWIST)
* nu: true anomaly         (ANGLE NOW)

****

Outputs:

- r: position vector (km)
- v: velocity vector (m/s)

r and v vectors are with respect to the geocentric equatorial frame

****

Other relations
- true longitude ≈ longitude of ascending node + argument of periapsis + true anomaly
- argument of latitude = argument of periapsis + true anomaly
- true longitude of periapsis ≈ longitude of ascending node + argument of periapsis 


In [149]:
import numpy as np
import scipy.optimize as scio
import matplotlib.pyplot as plt
import math

In [150]:
## Global variables and constants

# Standard gravitational parameter of Earth (3.986*10^14 m^3*s^-2)
# Multiplied by 10e-9 to convert to km^3*s^-2
mu = 3.986004418 * 10.0**14 * (10**(-9)) # km^3 / s^2

# I or x unit vector
I = np.array([1, 0, 0])

In [151]:
## Rotation functions

# Rotation around 1st component of 3-vector
# in x, y, z coordinates, this would rotation of system around x
def ROT1_func(rads):
    
    ROT1 = np.array([[1, 0, 0],
           [0, math.cos(rads), math.sin(rads)],
           [0, -math.sin(rads), math.cos(rads)]], dtype = object)
    
    return ROT1

# Rotation around 3rd component of 3-vector
# in x, y, z coordinates, this would rotation of system around z
def ROT3_func(rads):
    
    ROT3 = np.array([[math.cos(rads), math.sin(rads), 0],
           [-math.sin(rads), math.cos(rads), 0],
           [0, 0, 1]], dtype = object)
    
    return ROT3

In [152]:
# Special coordinates, used for special cases
def specialOE_func(p, e, i, capOmega, omega, nu):
    
    # Allocate memory
    specialOE = np.zeros(3)
    
    # True longtitude
    specialOE[0] = math.radians(capOmega + omega + nu)
    
    # Argument of longtitude
    specialOE[1] = math.radians(omega + nu)
    
    # True longtitude of periapsis
    specialOE[2] = math.radians(capOmega + omega)
    
    return specialOE

In [153]:
# Convert orbital elements to r and v in PQW coordinates
def elements_rv(p, e, i, capOmega, omega, nu, specialOE):
    
    # Allocate memory for vectors
    r_pqw = np.zeros(3)  
    v_pqw = np.zeros(3)
    
    # If Circular Equatorial
    if e == 0 and i == 0:
        omega, capOmega = 0.0
        nu = specialOE[0]
    
    # If Circular Inclined
    if e == 0 and i > 0:
        omega = 0.0
        nu = specialOE[1]
    
    # If Elliptical Equatorial
    if e > 0 and i == 0:
        capOmega = 0
        omega = specialOE[2]
        
    r_pqw[0] = (p*math.cos(nu))/(1+e*math.cos(nu))
    r_pqw[1] = (p*math.sin(nu))/(1+e*math.cos(nu))
    r_pqw[2] = 0
    
    v_pqw[0] = -((mu/p)**(1/2))*math.sin(nu)
    v_pqw[1] = ((mu/p)**(1/2))*(e+math.cos(nu))
    v_pqw[2] = 0
    
    return r_pqw, v_pqw

In [157]:
# Convert r and v vectors in PQW coordinates to IJK coordinates
# in the geocentric equatorial frame
def PQW_ijk(r_pqw, v_pqw):
    
    A = ROT3_func(-capOmega)
    B = ROT1_func(-i)
    C = ROT3_func(-omega)
    
    IJK_PQW = np.matmul(A, np.matmul(B, C))
    
    r_ijk = np.matmul(IJK_PQW, r_pqw)
    v_ijk = np.matmul(IJK_PQW, v_pqw)
    
    return r_ijk, v_ijk

In [158]:
def print_rv(r, v):
    
    r = list(r)
    v = list(v)
    
    for i, component in enumerate(r):
        r[i] = float("{:.3f}".format(component))
    
    for i, component in enumerate(v):
        v[i] = float("{:.3f}".format(component))
    
    print("Geocentric equatorial position vector: ")
    for component in r:
        print("  " + str(component) + " km")
        
    print(" ")
    print("Geocentric equatorial velocity vector: ")
    for component in v:
        print("  " + str(component) + " km/s")

In [159]:
# Test case, taken from example 2-6 of Vallado (pg 119)
p = 11067.790
e = 0.83285

# Orbital elements in degrees converted to radians
# as trig functions from math library take in only radians
i = math.radians(87.87)
capOmega = math.radians(227.89)
omega = math.radians(53.38)
nu = math.radians(92.335)

specialOE = specialOE_func(p, e, i, capOmega, omega, nu)
r_pqw, v_pqw = elements_rv(p, e, i, capOmega, omega, nu, specialOE)
r_ijk, v_ijk = PQW_ijk(r_pqw, v_pqw)

print_rv(r_ijk, v_ijk)

Geocentric equatorial position vector: 
  6525.368 km
  6861.532 km
  6449.119 km
 
Geocentric equatorial velocity vector: 
  4.902 km/s
  5.533 km/s
  -1.976 km/s
